[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcoaleixo/aisummit/blob/master/AISummit%20-%20machine_translation.ipynb)

# Seq2Seq para tradução inglês - Português

Modelo seq2seq simples para tradução de texto de inglês para português com leitura character por character.

In [0]:
#executar apenas uma vez
!pip3 install --upgrade torch torchvision googledrivedownloader

from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1_NiGFhSifCeu5_EEAbCIAh9rzSl1kXPG', dest_path='./por.txt', unzip=False)

In [0]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
import numpy as np
from google.colab import files

In [0]:
data_path = 'por.txt' #Arquivo com pares inglês-português
latent_dim = 264  #Tamanho do estado h
num_samples = 10000  # Numero de amostras a serem usadas

Aqui vamos carregar, fazer o um pré processamento e visualização do nosso dataset 

In [0]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

lines = open(data_path, encoding="utf8").read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # Usaremos um \t para indicar o inicio de uma frase e \n para indicar o fim
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Número de amostras:', len(input_texts))
print('Número de tokens unicos de input:', num_encoder_tokens)
print('Number de tokens unicos de output:', num_decoder_tokens)
print('Tamanho máximo de input:', max_encoder_seq_length)
print('Tamanho máximo de output:', max_decoder_seq_length)


Aqui vamos criar o dicionário de tokens para podermos fazer a conversão para texto futuramente

In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

print()
print('input_char: ', input_characters)
print()
print('input_token_index: ', input_token_index)

In [0]:
#cria arrays de 0 para fazer o padding
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

#preenche os arrays com os caracteres
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data é a saída esperada do decoder, então deve estar um tempo a frente e não incluir o \t
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

#one hot encoding
print(input_texts[0])
for i in encoder_input_data[0]:
    print(i)

## Seq2Seq

Aqui criamos um modelo Seq2Seq simples com uma camada de LSTM como encoder e outra como decoder.

O encoder retorna seus estados que serão a inicialização do decoder



In [0]:
"""
Teacher Forcing:

Suponha a frase inicial:
Mary had a little lamb whose fleece was white as snow

A mesma frase com os marcadores de inicio e fim:
[START] Mary had a little lamb whose fleece was white as snow [END]

Treinamento:

X                        y
--------------------------
[START],                 ?
[START], Mary,           ?
[START], Mary, had,      ?
[START], Mary, had, a,   ?
...

a diferença para a inferencia é que no treinamento durante a geração da saida, 
o decoder é realimentado com a saida esperada e não a saida gerada.

"""
print()

In [0]:
from keras.models import Sequential, Model


encoder_inputs = Input(shape=(None, num_encoder_tokens))

#Encoder
encoder = LSTM(latent_dim, return_state=True)

#pega saida do encoder
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c] #mantem apenas os estados internos


decoder_inputs = Input(shape=(None, num_decoder_tokens))

#Decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

#pega saída do decoder
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states) #inicializa os estados interno do decoder com os do encoder

#camada Dense com softmax do tamanho do vocabulario para definir qual o caracter de saída
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

#O modelo para o treino tem como entrada o input a ser codificado e o input do Force Teaching do decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
## word-level model with integer sequences
# anteriormente deve-se separar por palavras e nao caracteres

#from keras.models import Sequential, Model

## Define an input sequence and process it.
#encoder_inputs = Input(shape=(None,))
#x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
#x, state_h, state_c = LSTM(latent_dim,
#                           return_state=True)(x)
#encoder_states = [state_h, state_c]

## Set up the decoder, using `encoder_states` as initial state.
#decoder_inputs = Input(shape=(None,))
#x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
#x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
#decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
#model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.summary()

Uma das formas mais elegantes de salvar os pesos e o modelo é via callbacks:

In [0]:
from keras.callbacks import ModelCheckpoint

# checkpoint
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
filepath = "seq2seq_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
batch_size = 64
epochs = 1

# Executa o treino
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs, shuffle=False, #Sem shuffle para não misturar frases de tamanhos diferentes
          validation_split=0.1, verbose = 1, callbacks=callbacks_list)

In [0]:
#uma forma mais simples de salvar os pesos é usando a função save_weights, que salvará o ultimo estado da rede
#model.save_weights('seq2seq_weights.h5')
#ou salva o modelo todo utilizando save
model.save('s2s_model.h5')

In [0]:
#Pesos do treino já executado
model.load_weights('seq2seq_weights.h5')
#carrega tudo: peso e modelo
#mode.load('s2s_model.h5')

A principal diferença entre o modelo de testes e o de treinamento é que no treino utilizamos o force teaching e no teste iremos usar a real saída do decoder como input

In [0]:
#Modelo do Encoder
encoder_model = Model(encoder_inputs, encoder_states)

#Inputs para o decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#Reutiliza a LSTM treinada anteriormente com entradas novas
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

#Agora nosso modelo possui como entrada os estados que irão ser gerados pelo encoder
#e o output do decoder gerado pelos tempos anteriores

#Como saída temos também os estados para garantir que no próximo tempo o Decoder continue de onde parou
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
#Cria dicionário de tokens inverso para decodificar a saída
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

A função decode_sequence abaixo irá gerar a saída do decoder, ela é diferente da rede de treino pois agora a cada passo realimentamos com a ultima saída gerada (e nao a ultima saída esperada)

In [0]:
def decode_sequence(input_seq):
    # Aplica as entradas no encoder para pegar seus estados (sua memória)
    states_value = encoder_model.predict(input_seq)

    # Gera um vetor de entrada nulo para o decoder
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # coloca o \t na primeira posição do vetor indicando para o decoder que deve iniciar a gerar texto
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Faz o loop gerando saídas do Decoder
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # gere novos tokens enquanto nao encontrar uma condição de parada 
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        # Pega o token de saída
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Traduz para texto
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Adiciona à string de resposta
        decoded_sentence += sampled_char

        # Se a saída for \n ou tiver atingido o tamanho máximo, para o loop
        if (sampled_char == '\n' or
               len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Recria o input do decoder com a última saída
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [0]:
#teste

for seq_index in range(10000,10010):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Abaixo temos a função de teste onde dado um texto em ingês ele irá retornar a tradução em português

In [0]:
def teste(teste):
    texto_seq = np.zeros((1,max_encoder_seq_length,num_encoder_tokens))

    for t, char in enumerate(teste):
        texto_seq[0, t, input_token_index[char]] = 1.
    
    decoded_sentence = decode_sequence(texto_seq)
    print('-')
    print('Input:', teste)
    print('Output:', decoded_sentence)

In [0]:
print(max_encoder_seq_length)
print(num_encoder_tokens)
teste('Have fun and be')